<a href="https://colab.research.google.com/github/Vinayak-Sharma12/Toxic-Comment-Classifier/blob/main/Toxic_Comment_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTING IMPORTANT LIBRARIES**

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from sklearn import preprocessing, decomposition, model_selection, metrics
from keras.preprocessing import sequence, text


# **DATA PREPROCESSING**

In [9]:
train = pd.read_csv('/content/train (3).csv')


We will drop the other columns and approach this problem as a Binary Classification Problem and also we will have our exercise done on a smaller subsection of the dataset(only 12000 data points) to make it easier to train the models

In [10]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)

In [11]:
train = train.loc[:12000,:]
train.shape

(12001, 3)

We will check the maximum number of words that can be present in a comment , this will help us in padding later

In [12]:
train['comment_text'].apply(lambda x:len(str(x).split())).max()

1403

Writing a function for getting auc score for validation

In [13]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''

    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

### Data Preparation

In [14]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values,
                                                  stratify=train.toxic.values,
                                                  random_state=42,
                                                  test_size=0.2, shuffle=True)

In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 1500
token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
#zero pad the sequences
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)
word_index = token.word_index

# **MODEL TRAINING**

In [40]:
%%time
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(1, activation='sigmoid'))
    model.add(Dropout(0.3))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1500, 300)         13049100  
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 100)               40100     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
 dropout (Dropout)           (None, 1)                 0         
                                                                 
Total params: 13089301 (49.93 MB)
Trainable params: 13089301 (49.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
CPU times: user 118 ms, sys: 3.75 ms, total: 122 ms
Wall time: 123 ms


In [41]:
model.fit(xtrain_pad, ytrain, epochs=3, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/3
150/150 [==============================] - 192s 1s/step - loss: 0.6662 - accuracy: 0.8952
Epoch 2/3
150/150 [==============================] - 189s 1s/step - loss: 0.5927 - accuracy: 0.9279
Epoch 3/3
150/150 [==============================] - 185s 1s/step - loss: 0.5023 - accuracy: 0.9476


In [44]:
model.fit(xtrain_pad, ytrain, epochs=2, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/2
150/150 [==============================] - 184s 1s/step - loss: 0.5169 - accuracy: 0.9580
Epoch 2/2
150/150 [==============================] - 182s 1s/step - loss: 0.5117 - accuracy: 0.9607


# **ACCURACY**

In [45]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

76/76 [==============================] - 8s 103ms/step
Auc: 0.78%


In [46]:
scores_model = []
scores_model.append({'Model': 'SimpleRNN','AUC_Score': roc_auc(scores,yvalid)})

# **PREDICTION**

In [70]:
import numpy as np

# Assuming `model` is your trained model
# Assuming `tokenizer` is the tokenizer you used for preprocessing your text data

# Text to predict
text1 = "you bastard"
text2="I like your confindence"
text3="You are a bitch"
text4="I appreciate your behaviour"
text5="You are a peice of a shit"
text6="I mean look how adorable you are"




# Tokenize the text
tokenized_text1= token.texts_to_sequences([text1])
tokenized_text2= token.texts_to_sequences([text2])
tokenized_text3 = token.texts_to_sequences([text3])
tokenized_text4 = token.texts_to_sequences([text4])
tokenized_text5 = token.texts_to_sequences([text5])
tokenized_text6 = token.texts_to_sequences([text6])

# Pad sequences to a fixed length (if necessary)
max_sequence_length = 100  # Example sequence length
padded_sequence1 = pad_sequences(tokenized_text1, maxlen=max_sequence_length)
padded_sequence2 = pad_sequences(tokenized_text2, maxlen=max_sequence_length)
padded_sequence3 = pad_sequences(tokenized_text3, maxlen=max_sequence_length)
padded_sequence4 = pad_sequences(tokenized_text4, maxlen=max_sequence_length)
padded_sequence5 = pad_sequences(tokenized_text5, maxlen=max_sequence_length)
padded_sequence6 = pad_sequences(tokenized_text6, maxlen=max_sequence_length)

# Make predictions
predictions1 = model.predict(padded_sequence1)
predictions2 = model.predict(padded_sequence2)
predictions3 = model.predict(padded_sequence3)
predictions4 = model.predict(padded_sequence4)
predictions5 = model.predict(padded_sequence5)
predictions6 = model.predict(padded_sequence6)
if predictions1>0.5:
   print('Toxic')
else:
  print('Not Toxic')

if predictions2>0.5:
   print('Toxic')
else:
  print('Not Toxic')

if predictions3>0.5:
   print('Toxic')
else:
  print('Not Toxic')

if predictions4>0.5:
   print('Toxic')
else:
  print('Not Toxic')

if predictions5>0.5:
   print('Toxic')
else:
  print('Not Toxic')

if predictions6>0.5:
   print('Toxic')
else:
  print('Not Toxic')



1/1 [==============================] - 0s 23ms/step
Toxic
Not Toxic
Toxic
Not Toxic
Toxic
Not Toxic
